In [60]:
import astropy
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from matplotlib.pyplot import figure

In [61]:
#APOKASC file & file variables
hdul = fits.open(r"/Users/amber3/Desktop/SSI/APOKASC_cat_v6.7.2.fits")
#hdul.info()
hdu = hdul[1]
data = hdu.data
#print(np.shape(data))
KIC_A = hdu.data['KEPLER_INT'] 
Logg_A = hdu.data['DR16_LOGG_COR']
Logg_err = hdu.data['DR16_LOGG_COR_ERR']
Teff_A = hdu.data['DR16_TEFF_COR']
Teff_err = hdu.data['DR16_TEFF_COR_ERR']
FeH_A = hdu.data['DR16_M_H_COR']
FeH_A_err = hdu.data['Dr16_M_H_COR_ERR']
#print(Logg_A)
FeH_err = hdu.data['DR16_M_H_COR_ERR']
R_APO3 = hdu.data['APOKASC3P_RADIUS']
R_APO3_err = hdu.data['APOKASC3P_RADIUS_ERR']
R_Gaia = hdu.data['GAIA_RADIUS'] #no error on this radius
M_APO3 = hdu.data['APOKASC3P_MASS']
M_APO3_err = hdu.data['APOKASC3P_MASS_ERR']

In [62]:
#APOLLINAIRE file & file variables (b = background)
hdul_b = fits.open(r"/Users/amber3/Desktop/SSI/Selected_stars_final.fits")
hdu_b = hdul_b[0]
data_b = hdu_b.data
KIC_b = np.array(data_b[:,0,0])
#print(KIC_b)

In [81]:
common_KIC, A_indices, b_indices = np.intersect1d(KIC_A, KIC_b, return_indices=True)
#print(np.shape(A_indices), np.shape(b_indices))
good_indices = np.where(Teff_A[A_indices]*R_APO3[A_indices] > 0)
print(good_indices)
KIC_b = KIC_b[b_indices]
b_indices = b_indices[good_indices]
A_indices = A_indices[good_indices]
print(np.where(Teff_A[A_indices] < 0))
print(A_indices) #this seems to be working but then it doesn't below

Amp_HV1 = np.array(data_b[b_indices,0,6])
Amp_HV1_err = np.array(data_b[b_indices,1,6])
Amp_HV2 = np.array(data_b[b_indices,0,9])
Amp_HV2_err = np.array(data_b[b_indices,1,9])
Freq_HV1 = np.array(data_b[b_indices,0,7])
Freq_HV1_err = np.array(data_b[b_indices,1,7])
Freq_HV2 = np.array(data_b[b_indices,0,10])
Freq_HV2_err = np.array(data_b[b_indices,1,10])
Evol_stage = np.array(data_b[b_indices,0,3])
#print(KIC_A, KIC_b)

(array([   0,    1,    2, ..., 3997, 3998, 3999]),)
(array([ 318,  761,  794, 2155, 2210, 3249, 3296, 3400, 3604]),)
[ 863 4166 4232 ... 5782 5784 5795]


In [80]:
R_APO3_final = R_APO3[A_indices]
R_APO3_final_err = R_APO3_err[A_indices]
R_Gaia_final = R_Gaia[A_indices] #no err yet
#print(np.shape(Amp_HV1), np.shape(R_APO3_final))#bruh

logg_final = Logg_A[A_indices]
logg_final_err = Logg_err[A_indices]
Teff_final = Teff_A[A_indices]
print(Teff_final)
print(np.where(Teff_final < 0)) #why did this not change? A_indices is defined above
log_Teff_final = np.log10(Teff_final)
Teff_final_err = Teff_err[A_indices]
Teff_err_log = ((Teff_final_err)/(log_Teff_final))/np.log(10) 
FeH_final = FeH_A[A_indices]
FeH_final_err = FeH_A_err[A_indices]
#print(log_Teff_final) #runtime warning with neg values?
#print(Teff_err_log)

rpw = 2

Amp_HV1_APO3 =  Amp_HV1 * R_APO3_final**rpw
Amp_HV1_APO3_err = ((Amp_HV1_err/Amp_HV1) + (rpw*R_APO3_final_err/R_APO3_final)) * Amp_HV1_APO3
Amp_HV1_APO3_err_log = ((Amp_HV1_APO3_err)/(Amp_HV1_APO3))/np.log(10)

Amp_HV2_APO3 =  Amp_HV2 * R_APO3_final**rpw
Amp_HV2_APO3_err = ((Amp_HV2_err/Amp_HV2) + (rpw*R_APO3_final_err/R_APO3_final)) * Amp_HV2_APO3
Amp_HV2_APO3_err_log = ((Amp_HV2_APO3_err)/(Amp_HV2_APO3))/np.log(10)

Amp_HV1_Gaia =  Amp_HV1 * R_Gaia_final**rpw
#no err

Amp_HV2_Gaia =  Amp_HV2 * R_Gaia_final**rpw
#no err

Amp_HV1_err_log = ((Amp_HV1_err)/(Amp_HV1))/np.log(10)
Amp_HV2_err_log = ((Amp_HV2_err)/(Amp_HV2))/np.log(10)
Freq_HV1_err_log = ((Freq_HV1_err)/(Freq_HV1))/np.log(10)
Freq_HV2_err_log = ((Freq_HV2_err)/(Freq_HV2))/np.log(10)

[4635.539  4786.9126 4781.582  ... 4898.131  4992.812  4641.728 ]
(array([ 318,  761,  794, 2155, 2210, 3249, 3296, 3400, 3604]),)


/var/folders/yk/s_2b7kzx717dbt8bm2wpx7qr0000gr/T/ipykernel_6435/2901714727.py:11: RuntimeWarning: invalid value encountered in log10
  log_Teff_final = np.log10(Teff_final)


In [20]:
#%matplotlib inline
def freq_curve_fit(X, a, b, c, d):
    x,y,z = X
    return a+b*x+c*y+z*d

def logg_fit(X, a, b):
    x = X
    return a + b*x
         
red_giant = np.where(Evol_stage == 1)
red_clump = np.where(Evol_stage == 2)
#print(Teff_final[red_giant])

mean_logg = np.mean(logg_final[red_giant])
mean_l_teff = np.mean(log_Teff_final[red_giant])
print(mean_l_teff) #neg value in log_Teff_final bc of runtime warning in Teff_final, printing nan

logg_opt, logg_cov = curve_fit(logg_fit, (np.log10(Teff_final[red_giant])-mean_l_teff), logg_final[red_giant]-mean_logg, sigma=(logg_final_err[red_giant]))
logg_perr = np.sqrt(np.diag(logg_cov))
logg_opt[1] = 18.614913/0.7

theta = ((np.pi/2) - np.arctan2(logg_opt[1], 1))
#rotated plots
x_prime = ((np.log10(Teff_final)-mean_l_teff)*np.cos(theta)) - ((logg_final-mean_logg)*np.sin(theta))
y_prime = ((np.log10(Teff_final)-mean_l_teff)*np.sin(theta)) + ((logg_final-mean_logg)*np.cos(theta))
coeffs = np.ones((4, 6, 2)) #switch 2 and 4?
coeffs_err = np.ones((4, 6, 2))

nan


/var/folders/yk/s_2b7kzx717dbt8bm2wpx7qr0000gr/T/ipykernel_6435/3176269612.py:18: RuntimeWarning: invalid value encountered in log10
  logg_opt, logg_cov = curve_fit(logg_fit, (np.log10(Teff_final[red_giant])-mean_l_teff), logg_final[red_giant]-mean_logg, sigma=(logg_final_err[red_giant]))


ValueError: array must not contain infs or NaNs

In [ ]:
#fit in rotated atm params with respect to APO3 R corrected amps

#amplitude 1 red giants
A1_Rg_x_opt, A1_Rg_x_cov = curve_fit(freq_curve_fit, (x_prime[red_giant], y_prime[red_giant], FeH_final[red_giant]), np.log10(Amp_HV1_APO3[red_giant]), sigma=(Amp_HV1_APO3_err_log[red_giant]))
A1_Rg_x_perr = np.sqrt(np.diag(A1_Rg_x_cov))
coeffs[:,4,0] = A1_Rg_x_opt
coeffs_err[:,4,0] = A1_Rg_x_perr
print('A1_RG')
print(A1_Rg_x_opt)
print(A1_Rg_x_perr)

#amplitude 1 red clump
A1_Rc_x_opt, A1_Rc_x_cov = curve_fit(freq_curve_fit, (x_prime[red_clump], y_prime[red_clump], FeH_final[red_clump]), np.log10(Amp_HV1_APO3[red_clump]), sigma=(Amp_HV1_APO3_err_log[red_clump]))
A1_Rc_x_perr = np.sqrt(np.diag(A1_Rc_x_cov))
coeffs[:,4,1] = A1_Rc_x_opt
coeffs_err[:,4,1] = A1_Rc_x_perr
print('A1_RC')
print(A1_Rc_x_opt)
print(A1_Rc_x_perr)

#amplitude 2 red giants
A2_Rg_x_opt, A2_Rg_x_cov = curve_fit(freq_curve_fit, (x_prime[red_giant], y_prime[red_giant], FeH_final[red_giant]), np.log10(Amp_HV2_APO3[red_giant]), sigma=(Amp_HV2_APO3_err_log[red_giant]))
A2_Rg_x_perr = np.sqrt(np.diag(A2_Rg_x_cov))
coeffs[:,5,0] = A2_Rg_x_opt
coeffs_err[:,5,0] = A2_Rg_x_perr
print('A2_RG')
print(A2_Rg_x_opt)
print(A2_Rg_x_perr)

#amplitude 2 red clump
A2_Rc_x_opt, A2_Rc_x_cov = curve_fit(freq_curve_fit, (x_prime[red_clump], y_prime[red_clump], FeH_final[red_clump]), np.log10(Amp_HV2_APO3[red_clump]), sigma=(Amp_HV2_APO3_err_log[red_clump]))
A2_Rc_x_perr = np.sqrt(np.diag(A2_Rc_x_cov))
coeffs[:,5,1] = A2_Rc_x_opt
coeffs_err[:,5,1] = A2_Rc_x_perr
print(np.shape(Amp_HV2_err_log[red_clump]))
print(np.shape(x_prime[red_clump]))
print('A2_RC')
print(A2_Rc_x_opt)
print(A2_Rc_x_perr)

In [ ]:
#fit in rotated atm params with respect to Gaia R corrected amps
#no sigma

#amplitude 1 red giants
A1_Rg_x_opt, A1_Rg_x_cov = curve_fit(freq_curve_fit, (x_prime[red_giant], y_prime[red_giant], FeH_final[red_giant]), np.log10(Amp_HV1_Gaia[red_giant]))
A1_Rg_x_perr = np.sqrt(np.diag(A1_Rg_x_cov))
coeffs[:,4,0] = A1_Rg_x_opt
coeffs_err[:,4,0] = A1_Rg_x_perr
print('A1_RG')
print(A1_Rg_x_opt)
print(A1_Rg_x_perr)

#amplitude 1 red clump
A1_Rc_x_opt, A1_Rc_x_cov = curve_fit(freq_curve_fit, (x_prime[red_clump], y_prime[red_clump], FeH_final[red_clump]), np.log10(Amp_HV1_Gaia[red_clump]))
A1_Rc_x_perr = np.sqrt(np.diag(A1_Rc_x_cov))
coeffs[:,4,1] = A1_Rc_x_opt
coeffs_err[:,4,1] = A1_Rc_x_perr
print('A1_RC')
print(A1_Rc_x_opt)
print(A1_Rc_x_perr)

#amplitude 2 red giants
A2_Rg_x_opt, A2_Rg_x_cov = curve_fit(freq_curve_fit, (x_prime[red_giant], y_prime[red_giant], FeH_final[red_giant]), np.log10(Amp_HV2_Gaia[red_giant]))
A2_Rg_x_perr = np.sqrt(np.diag(A2_Rg_x_cov))
coeffs[:,5,0] = A2_Rg_x_opt
coeffs_err[:,5,0] = A2_Rg_x_perr
print('A2_RG')
print(A2_Rg_x_opt)
print(A2_Rg_x_perr)

#amplitude 2 red clump
A2_Rc_x_opt, A2_Rc_x_cov = curve_fit(freq_curve_fit, (x_prime[red_clump], y_prime[red_clump], FeH_final[red_clump]), np.log10(Amp_HV2_Gaia[red_clump]))
A2_Rc_x_perr = np.sqrt(np.diag(A2_Rc_x_cov))
coeffs[:,5,1] = A2_Rc_x_opt
coeffs_err[:,5,1] = A2_Rc_x_perr
print(np.shape(Amp_HV2_err_log[red_clump]))
print(np.shape(x_prime[red_clump]))
print('A2_RC')
print(A2_Rc_x_opt)
print(A2_Rc_x_perr)

In [ ]:
N_RC = np.size(red_clump)
log_Teff_final_RG = log_Teff_final[red_giant]
log_Teff_final_RC = log_Teff_final[red_clump]
logg_final_RG = logg_final[red_giant]
logg_final_RC = logg_final[red_clump]
FeH_final_RG = FeH_final[red_giant]
FeH_final_RC = FeH_final[red_clump]
print(N_RC)
index_RC = ()
index_RG = ()
eps = 0.04
print(type(index_RC))
#print(type(red_giant))
for i in range(0, N_RC): 
    dist = np.sqrt((7*(log_Teff_final_RG-log_Teff_final_RC[i]))**2 + (logg_final_RG-logg_final_RC[i])**2 + (3*(FeH_final_RG-FeH_final_RC[i]))**2)
    min_dist = (np.min(dist))
    if min_dist < eps:
        index = np.where(dist == min_dist)
        index_RC = index_RC + (i,)
        #index_RC = np.append(index_RC, i)
        index_RG = index_RG + (index[0],)
        #print(index[0])
print(np.size(index_RC), np.size(index_RG))
print(index_RG)
np.shape(index_RG)